In [1]:
from pdfminer.high_level import extract_text

text = extract_text("dataset/book.pdf")
print(text)

Psychology 2e 

SENIOR CONTRIBUTING AUTHORS 
ROSE M. SPIELMAN, FORMERLY OF QUINNIPIAC UNIVERSITY 
WILLIAM J. JENKINS, MERCER UNIVERSITY 
MARILYN D. LOVETT, SPELMAN COLLEGE 

	
	
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
OpenStax 
Rice University 
6100 Main Street MS-375 
Houston, Texas 77005 

To learn more about OpenStax, visit https://openstax.org. 
Individual print copies and bulk orders can be purchased through our website. 

©2020 Rice University. Textbook content produced by OpenStax is licensed under a Creative Commons 
Attribution 4.0 International License (CC BY 4.0). Under this license, any user of this textbook or the textbook 
contents herein must provide proper attribution as follows:  

-

-

-

-

If you redistribute this textbook in a digital format (including but not limited to PDF and HTML), then you
must retain on every page the following attribution:
“Access for free at openstax.org.”
If you redistribute this textbook in a print format, then you must include 